<h1 style="text-align:center">Shell.AI EV Problem (LP part) </h1>

In [ ]:
!pip install pulp

In [2]:
from pulp import *
import numpy as np
import pandas as pd

In [3]:
Dist = np.random.randn(4096,100)

In [4]:
# so we have here is existing infrastructure and we need to model the different data points(supply) 
#that goes with the demand points
supply_points_existing = pd.read_csv("./exisiting_EV_infrastructure_2018.csv")
supply_points_existing

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,3
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4
96,96,30.318396,33.388335,32,5,10
97,97,36.218839,22.235766,32,4,14
98,98,42.936915,38.122442,28,7,5


In [5]:
demand=pd.read_csv("./Demand_History.csv")
demand.head()

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563


In [6]:
demand_2018 = demand["2018"].to_numpy()
np.sum(demand_2018) # 2018 EXISTING DEMAND

361529.6365968907

In [7]:
#distance matrix between demand points and supply points
xs=np.array(supply_points_existing["x_coordinate"])
ys=np.array(supply_points_existing["y_coordinate"])
xd=np.array(demand["x_coordinate"])
yd=np.array(demand["y_coordinate"])
dist=np.zeros((100,4096))

for i in range(100):
    for j in range(4096):
        dist[i][j]=(((xs[i]-xd[j])**2)+((ys[i]-yd[j])**2))**0.5

dist=dist.T

In [8]:
SCS = supply_points_existing["existing_num_SCS"].to_numpy()
FCS = supply_points_existing["existing_num_FCS"].to_numpy()
parking_slots = supply_points_existing["total_parking_slots"].to_numpy()

In [9]:
np.sum(200*SCS + 400*FCS) # EXISTING SUPPLY

361600

In [54]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The EV problem", LpMinimize)

# setting up the required LP variables in the problems
supply_vars_SCS = LpVariable.dict("SCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero(constraint - 1)
supply_vars_FCS = LpVariable.dict("FCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero ( constraint - 1)
demand_supply_matrix = LpVariable.dict("Demand", (range(4096),range(100)), 0,None, cat=LpContinuous) 
# demand is varying continuously as given in the data (constraint 1+2)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
np.size(dist)

409600

In [ ]:
demand_supply_matrix[0,1].value()

In [56]:
# The objective function is added to 'prob' first
prob += (
    lpSum([
          [
            (dist[i][j]*(demand_supply_matrix[i,j])) 
            for i in range(4096)] for j in range(100)
         ]
    )+
        lpSum([
          
            ((supply_vars_SCS[j] + 1.5*supply_vars_FCS[j])) 
             for j in range(100)
         ]
        ), 
    "Total Cost Function",
)

In [58]:
# 3) Sum of FCS and SCS at jth point must be less than the total parking slots at the point
for j in range(100):
  prob += (
    lpSum(supply_vars_SCS[j] + supply_vars_FCS[j]) <= parking_slots[j],
    f"Parking_max {j}",
  )

In [59]:
# Now we are defining the constraints to the problem here that are needed to be solved 
# 4) the infrastructure needs to be built on the existing infrastructure
for j in range(100):
  prob += (
    supply_vars_SCS[j] >= SCS[j],
    f"SCS_min_{j}",
  ) 
for j in range(100):
  prob += (
    supply_vars_FCS[j] >= FCS[j],
    f"FCS_min_{j}",
  )

In [60]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
for j in range(100):
  prob += (
   lpSum([demand_supply_matrix[i,j] for i in range(4096)]) - 
   lpSum(200*supply_vars_SCS[j] + 400*supply_vars_FCS[j]) <= 0,
   f"demand_{j}",
  ) 

In [61]:
# 6) Demand satisfed by each ith point should equal to the demand at the time
for i in range(4096):
  prob += (
    lpSum([demand_supply_matrix[i,j] for j in range(100)]) == demand_2018[i],
    f"demand_match_{i}",
  ) 

In [62]:
prob.solve()

1

In [63]:
demand_supply_matrix_2018 = np.zeros((4096,100))
for i in range(4096):
  for j in range(100):
    demand_supply_matrix_2018[i][j] = demand_supply_matrix[i,j].value()

In [64]:
np.sum(demand_supply_matrix_2018)

361529.63658368995

In [65]:
SCS_2018 = np.zeros(100)
FCS_2018 = np.zeros(100)
for i in range(100):
    SCS_2018[i] = supply_vars_SCS[i].value()
for i in range(100):
    FCS_2018[i] = supply_vars_FCS[i].value()

In [66]:
np.sum(200*SCS_2018 + 400*FCS_2018)

479800.0

In [67]:
# Value of cost function I got here
cost_2018 = 0
for i in range(4096):
  for j in range(100):
      cost_2018 += (dist[i][j]*(demand_supply_matrix_2018[i][j])) 
for j in range(100):
  cost_2018 += 600*(SCS_2018[j] + 1.5*FCS_2018[j])


In [68]:
cost_2018

2349355.8137419047

In [71]:
score_2018 = max(10, 100 - ((90*cost_2018)/15000000))
score_2018

85.90386511754858